In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('Data/ml-1m/movies.dat', delimiter='\t', encoding='latin', names=['movieId'])
users = pd.read_csv('Data/ml-1m/users.dat', names=['userId'])
ratings = pd.read_csv('Data/ml-1m/ratings.dat', names=['userId'])

In [ ]:
movies[['movieId', 'title', 'genre']] = movies['movieId'].apply(lambda x: pd.Series(str(x).split('::')))
users[['userId','Gender','Age','Occupation','Zip-code']] = users['userId'].apply(lambda x: pd.Series(str(x).split('::')))
ratings[['userId','movieId','Rating','Timestamp']] = ratings['userId'].apply(lambda x: pd.Series(str(x).split('::')))

In [ ]:
movie_ratings = pd.merge(movies, ratings, on='movieId')
movie_user_rating = pd.merge(movie_ratings, users, on='userId')

In [ ]:
genre = movie_user_rating['genre'].str.get_dummies('|')

In [ ]:
movie_user_rating = movie_user_rating.join(genre)

In [ ]:
movie_user_rating[['Rating']] = movie_user_rating[['Rating']].apply(pd.to_numeric)

In [ ]:
movie_user_rating.drop(columns=['genre', 'title', 'Timestamp','Zip-code'], inplace=True)

In [ ]:
movie_user_rating['Gender'] = movie_user_rating["Gender"].map({"M": 1, "F": 0})

To get the average rating of every user to each genre

In [ ]:
genres = ['Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

for i in genres:
    movie_user_rating[i + '_rating'] = movie_user_rating[i]*movie_user_rating['Rating']
movie_user_rating.drop(columns=genres, inplace=True)
mean_ratings = movie_user_rating.groupby(by='userId').mean()

Machine learning model to predict ratings

In [ ]:
movie_user_rating.drop(columns=['userId','movieId'], inplace=True)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

x = movie_user_rating.drop(columns=['Rating'])
y = movie_user_rating['Rating']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

K Nearest Neighbour Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

k = 5  # Number of neighbors
knn_model = KNeighborsRegressor(n_neighbors=k)

knn_model.fit(X_train_scaled, y_train)

y_pred = knn_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
import pickle
filename = 'knnRegression.sav'
pickle.dump(knn_model, open(filename, 'wb'))